In [189]:
# !pip install openpyxl
# !pip install selenium
# !pip install webdriver_manager

In [190]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # 크롬버전 상관없이 드라이버 설치 상관없이 
from selenium.webdriver.chrome.service import Service # 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from openpyxl import load_workbook

import pyperclip
import time
import pandas as pd
import numpy as np
import csv

In [191]:
#-- activate webdriver 
    # browser = webdriver.Chrome("../chromedriver.exe")
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [192]:
#-- LogIn
def login_service(id,pwd):
    browser.get("https://nxlogin.nexon.com/common/login.aspx?redirect=https%3A%2F%2Fevents.fifaonline4.nexon.com%2F220908%2Fshoot%3Futm_source%3Dpc%26utm_medium%3DmainMiddle%26utm_campaign%3D220908_shoot")
    browser.implicitly_wait(1)
    search_id = browser.find_element(By.CSS_SELECTOR,"#txtNexonID")
    search_id.click()
    # search_id.send_keys(id)
    pyperclip.copy(id)
    search_id.send_keys(Keys.CONTROL,"v")
    time.sleep(1)
    search_pwd = browser.find_element(By.CSS_SELECTOR,"#txtPWD")
    search_pwd.click()
    # search_pwd.send_keys(pwd)
    pyperclip.copy(pwd)
    time.sleep(2)
    search_pwd.send_keys(Keys.CONTROL,"v")
    search_pwd.send_keys(Keys.ENTER)
    browser.implicitly_wait(3)
    

    try :    
        browser.find_element(By.CSS_SELECTOR,"#header > div.menu > button.btn_logout") 
        message = id + "\tLogIn Success"
    except:
        message = id + "\tLogIn Failed"
        browser.quit()
    print(message)


In [193]:
#-- LogOut
def logout():
    try:
        logout = browser.find_element(By.CSS_SELECTOR,"#header > div.menu > button.btn_logout")
        logout.click()
        print("LogOut complete")
    except NoSuchElementException as e : 
        print("Logout button not found")
    browser.implicitly_wait(3)

In [194]:
def refresh_btn():
    rb = browser.find_element(By.CSS_SELECTOR,"#middle > section > div.summer33_wrap > div > div.header > div > div > button.btn_check")
    rb.click()

In [195]:
def mission_data_upload(x):
    mission_list_name = []
    mission_list_data = []
    for i in range(1,7):
        mission_list_name.append(browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.txt".format(str(i))).text.replace(" ",""))
        mission_list_data.append(browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.state_txt > strong".format(str(i))).text.replace(" ","")+
                                 browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.state_txt > span".format(str(i))).text.replace(" ",""))
    df.loc[2*x] = mission_list_name
    df.loc[2*x+1] = mission_list_data
    return df

In [ ]:
#-- id / pwd load
id_data = pd.read_csv("../../../id_data.csv")
id_count = id_data.shape[0]
mission_count = 6
df = pd.DataFrame(np.zeros([id_count*2,mission_count]))
for x in list(range(id_count)):
    d1 = id_data["id"][x]
    d2 = id_data["pwd"][x]
    
    login_service(d1,d2)
    
    mission_data_upload(x)
    logout()
df
#-- logIn
